__A tutorial and template for *PersLay: A Simple and Versatile Neural Network Layer for Persistence Diagrams*.__

__Author:__ Théo Lacombe, Mathieu Carrière

__Note:__ This is an alpha version of PersLay. Do not hesitate to contact the authors for any comment, suggestion, bug, etc.

__Outline:__
In this notebook:
- First, we select a dataset. Two types of datasets are provided by default, either synthetic orbits from dynamical systems, or real-life graph dataset (we also explain how you could use PersLay with your own persistence diagrams).
- Then, we generate the persistence diagrams (and other useful informations such as labels, etc.) for the chosen dataset.
- (Optional) we propose to visualize the generated persistence diagrams.
- We define a neural network that uses some PersLay channels as first layers to handle persistence diagrams. This can be used as a guideline to use PersLay in your own experiments.
- We show how to train this neural network.

# Import required Python libraries

Print the current version of Python.

In [ ]:
import sys
print("Current version of your system: (we recommand Python 3.6)")
print(sys.version)

Import Numpy, TensorFlow, PersLay.

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import *
from sklearn.svm import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow import random_uniform_initializer as rui

In [ ]:
from perslay.perslay import perslay_channel
from perslay.preprocessing import preprocess
from perslay.visualisation import visualise_diag
from perslay.experiments   import load_diagfeatlabels, generate_diag_and_features, single_run, perform_expe

# (Optional) Generate predefined persistence diagrams

__Note:__ Skip this section and go to Section 3 if you already have your own persistence diagrams.

We start by choosing the dataset we want to run the experiments on. We suggest the user to start with `"MUTAG"` as this dataset is reasonably small (188 graphs with 18 nodes on average). Note that its small size implies a large variability in tests.

Available options are:

- Orbit datasets: `"ORBIT5K"`, `"ORBIT100K"`.

- Graphs datasets: `"BZR"`,`"FRANKENSTEIN"`,`"MUTAG"`,`"COX2"`, `"DHFR"`, `"PROTEINS"`, `"NCI1"`, `"NCI109"`,`"IMDB-BINARY"`, `"IMDB-MULTI"`.

__Important note:__ `"COLLAB"`,`"REDDIT5K"` and `"REDDIT12K"` are not available yet (see README.md). Contact the authors for more information.

Beware that for the larger datasets (`"COLLAB"`,`"REDDIT5K", "REDDIT12K", "ORBIT100K"`), the files can be quite large (e.g. 3Gb for for `"ORBIT100K"`), so that RAM can be limiting, and the time needed to generate the persistence diagrams and run the experiments can be quite long depending on the hardware available. Dataset descriptions are available in Section B of the supplementary material of the article.

In [ ]:
# Chose your config file using one of the filename mentioned above.
dataset = "PROTEINS"

Here, we implicitely load our data (saved as `.mat` files for graphs datasets, and generated on-the-fly for orbits datasets---which can take some time for `"ORBIT100K"` especially), and then compute the persistence diagrams that will be used in the classification experiment (requires to have `gudhi` installed). For graph datasets, we also generate a series of additional features (see [1]).

Running `generate_diag_and_features` will store diagrams, features and labels. Therefore, it is sufficient to run it just once (for each different dataset). Note that for bigger datasets, the computations of these persistence diagrams can be quite long.

In [ ]:
generate_diag_and_features(dataset)

##### Visualize persistence diagrams

Now we load and preprocess persistence diagrams (to make them PersLay-compatible) and other useful items using the files that we have generated.

In [ ]:
diags_tmp, feats, labels = load_diagfeatlabels(dataset)

Run the following cell to visualise some example of diagrams generated.

In [ ]:
visualise_diag(diags_tmp)

# (Optional) Use your own persistence diagrams

__Note:__ Skip this section and make sure to go through Section 2 if you want to use the predefined persistence diagrams that we provide.

We provide a (hopefully) convenient way to use your own persistence diagrams for a classification task (with some eventual features).

Persistence diagrams must be given in the following format:
assume you have $N$ observations. For each of them, you build $K$ different persistence diagrams (e.g. persistence diagrams in different homology dimensions, and/or for different filtrations, etc.). 

Then, you must provide a `diags_tmp` variable that is a `dictionary`, whose $K$ keys are the persistence diagram type names (e.g. `"Rips_dim_0"`, `"Cech_dim_1"`). For each key $k_i$, $1 \leq i \leq K$, the corresponding value is a `list` of `np.arrays`, each array encoding a persistence diagram. 

Note that each list must have the same length $N$ (you need to have the same number of persistence diagrams generated for each list). Note also that you must keep the order (i.e. the first element of each list must correspond to the persistence diagram generated with the first observation, and so on).

Below is an example of such a (very simple) dictionary, with two filtrations and two persistence diagrams in each:

`diags_tmp = {"Alpha0":[np.array([[0.1, 0.2], [0.2, 0.5], [0.3, 0.9]]), np.array([[0.1, 0.4], [0.3, 0.5]]),], "Alpha1":[np.array([[0.1, 0.4], [0.2, 0.6], [0.4, 0.9]]), np.array([[0.1, 0.2], [0.5, 0.7], [0.8, 0.9]])]}`

In [ ]:
### To use your own diagrams, uncomment and complete the following
#diags_tmp = ...

Now, we apply a preprocessing that makes our sets of persistence diagrams compatible with PersLay.

In [ ]:
import sklearn_tda as tda

### Uncomment the following to process your diagrams (necessary)
thresh = 500

# Whole pipeline
tmp = Pipeline([
        ("Selector",      tda.DiagramSelector(use=True, point_type="finite")),
        ("ProminentPts",  tda.ProminentPoints(use=True, num_pts=thresh)),
        ("Scaler",        tda.DiagramScaler(use=True, scalers=[([0,1], MinMaxScaler())])),
        ("Padding",       tda.Padding(use=True)),
                ])
prm = {filt: {"ProminentPts__num_pts": min(thresh, max([len(dgm) for dgm in diags_tmp[filt]]))} 
       for filt in diags_tmp.keys() if max([len(dgm) for dgm in diags_tmp[filt]]) > 0}

# Apply the previous pipeline on the different filtrations.
D = []
for dt in prm.keys():
    param = prm[dt]
    tmp.set_params(**param)
    D.append(tmp.fit_transform(diags_tmp[dt]))

# For each filtration, concatenate all diagrams in a single array.
diags = []
for dt in range(len(prm.keys())):
    diags.append(np.concatenate([D[dt][i][np.newaxis, :] for i in range(len(D[dt]))], axis=0))

Now, you must (obviously) provide the labels corresponding to each persistence diagram (be careful to keep the same order).

In [ ]:
### To use your own labels, uncomment and complete the following
#labels = ...

You can use some additional "standard" features in your network. These features must be provided as a $N \times d$ `np.array`, where $N$ is your number of observations (as before) and $d$ is the dimension of your features.

If you do not want to use additional features, you must use an empty array of size $(N,0)$, where $N$ is the number of observations.

In [ ]:
### Uncomment and complete the following line to not use feat.
#N = # number of observations
#feats = np.array([[]]*N)

### To use your own features instead, uncomment and complete the following
#feats = ...

# Using PersLay in a neural network

## (Optional) Define your own network

In case you use your own persistence diagrams, you might want to define your own architecture with PersLay. To help you with it, in the following section we define two (very simple) neural network architectures that use PersLay. They can be used as templates to build your own architecture.

### Set the hyper-parameters

#### Filtration parameters

In the following cell, you provide information on the filtrations used to generate the persistence diagrams. The only necessary keys to fill are `learn` (whether you want to optimize filtration values), `names` (names of your different filtrations) and `pad` (threshold value you used for padding the diagrams). Note that setting `learn` to `True`, i.e., optimizing over the filtration values, is not yet publicly available (hopefully it will be soon), so make sure `learn` is `False` for now.

In [ ]:
filt_parameters = {}
filt_parameters["learn"]      = False
filt_parameters["names"]      = [f for f in diags_tmp.keys()]
filt_parameters["pad"]        = thresh

This cell contains code referring to libraries that are not yet available, so do not run it for now.

In [ ]:
if filt_parameters["learn"]:
    filt_parameters["homology"]   = [[0]]
    filt_parameters["thresholds"] = [[500]]
    filt_parameters["init"]       = [[10]]

#### PersLay parameters

##### Layer type

Initialize dictionary of parameters for PersLay.

In [ ]:
perslay_parameters = {}

Choice of layer type, must be one of (see README.md and [1] for details):
- `"im"` for a persistence image layer.
- `"pm"` for a permutation equivariant layer (as in [2]).
- `"ex"` for an exponential structure element layer (as in [3]).
- `"rt"` for a rational structure element layer (as in [3]).
- `"rh"` for a rational hat structure element layer (as in [3]).
- `"ls"` for a persistence landscape layer.
- `"bc"` for a Betti curve layer.
- `"en"` for a persistence entropy layer.

In [ ]:
perslay_parameters["layer"]          = "im"
perslay_parameters["image_size"]     = (20, 20)
epsilon = .001
perslay_parameters["image_bnds"]     = ((0. - epsilon, 1. + epsilon), (0. - epsilon, 1. + epsilon))
perslay_parameters["variance_init"]  = rui(3.0, 3.0) 
perslay_parameters["variance_const"] = False
perslay_parameters["cv_layers"]      = []

In [ ]:
perslay_parameters["layer"]          = "pm"
perslay_parameters["peq"]            = [(5, "max")]
perslay_parameters["weight_init"]    = rui(0.0, 1.0)
perslay_parameters["weight_const"]   = False
perslay_parameters["bias_init"]      = rui(0.0, 1.0) 
perslay_parameters["bias_const"]     = False
perslay_parameters["fc_layers"]      = []

In [ ]:
perslay_parameters["layer"]          = "ex"
perslay_parameters["num_elements"]   = 25
perslay_parameters["mean_init"]      = rui(0.0, 1.0)
perslay_parameters["mean_const"]     = False
perslay_parameters["variance_init"]  = rui(3.0, 3.0) 
perslay_parameters["variance_const"] = False
perslay_parameters["fc_layers"]      = []

In [ ]:
perslay_parameters["layer"]          = "rt"
perslay_parameters["num_elements"]   = 25
perslay_parameters["mean_init"]      = rui(0.0, 1.0)
perslay_parameters["mean_const"]     = False
perslay_parameters["variance_init"]  = rui(3.0, 3.0) 
perslay_parameters["variance_const"] = False
perslay_parameters["alpha_init"]     = rui(3.0, 3.0) 
perslay_parameters["alpha_const"]    = False
perslay_parameters["fc_layers"]      = []

In [ ]:
perslay_parameters["layer"]          = "rh"
perslay_parameters["num_elements"]   = 25
perslay_parameters["mean_init"]      = rui(0.0, 1.0)
perslay_parameters["mean_const"]     = False
perslay_parameters["r_init"]         = rui(3.0, 3.0) 
perslay_parameters["r_const"]        = False
perslay_parameters["q"]              = 2
perslay_parameters["fc_layers"]      = []

In [ ]:
perslay_parameters["layer"]          = "ls"
perslay_parameters["num_samples"]    = 100
perslay_parameters["sample_init"]    = rui(0.0, 1.0) 
perslay_parameters["sample_const"]   = False
perslay_parameters["fc_layers"]      = []

In [ ]:
perslay_parameters["layer"]          = "bc"
perslay_parameters["theta"]          = 10
perslay_parameters["num_samples"]    = 100
perslay_parameters["sample_init"]    = rui(0.0, 1.0) 
perslay_parameters["sample_const"]   = False
perslay_parameters["fc_layers"]      = []

In [ ]:
perslay_parameters["layer"]          = "en"
perslay_parameters["theta"]          = 10
perslay_parameters["num_samples"]    = 100
perslay_parameters["sample_init"]    = rui(0.0, 1.0) 
perslay_parameters["sample_const"]   = False
perslay_parameters["fc_layers"]      = []

##### Weight function

Choice of the weight function, must be one of:
- `"linear"`, for a linear weight w.r.t. the distance to the diagonal.
- `"grid"`, for a piecewise-constant function defined with pixel values.
- `"gmix"`, for a weight function defined as a mixture of Gaussians.
- `None`, for a constant weight function. 

In [ ]:
perslay_parameters["persistence_weight"]  = "linear"
perslay_parameters["coeff_init"]          = rui(1.0, 1.0)
perslay_parameters["coeff_const"]         = False

In [ ]:
perslay_parameters["persistence_weight"]  = "grid"
perslay_parameters["grid_size"]           = [20,20]
epsilon = .001
perslay_parameters["grid_bnds"]           = ((0. - epsilon, 1. + epsilon), (0. - epsilon, 1. + epsilon))
perslay_parameters["grid_init"]           = rui(1.0, 1.0)
perslay_parameters["grid_const"]          = False

In [ ]:
perslay_parameters["persistence_weight"]  = "gmix"
perslay_parameters["gmix_num"]            = 3
perslay_parameters["gmix_m_init"]         = rui(0.0, 1.0)
perslay_parameters["gmix_m_const"]        = False
perslay_parameters["gmix_v_init"]         = rui(.01, .01)
perslay_parameters["gmix_v_const"]        = False

In [ ]:
perslay_parameters["persistence_weight"]  = None

##### Permutation-invariant operation

Choice of permutation invariant operator, must be one of:
- `"sum"`.
- `"topk"`, will select the $k$ highest values, specified in `keep`.
- `"max"`.
- `"mean"`.

In [ ]:
perslay_parameters["perm_op"] = "sum"

In [ ]:
perslay_parameters["perm_op"] = "topk"
perslay_parameters["keep"]    = 5

In [ ]:
perslay_parameters["perm_op"] = "max"

In [ ]:
perslay_parameters["perm_op"] = "mean"

##### Predefined layers

These set of predefined parameters reproduce the usual persistence images and landscapes from the literature, as well as the DeepSet vectorization.

Persistence image layer.

In [ ]:
PI_perslay_parameters                        = {}
PI_perslay_parameters["layer"]               = "im"
PI_perslay_parameters["image_size"]          = (10, 10)
epsilon = .001
PI_perslay_parameters["image_bnds"]          = ((0. - epsilon, 1. + epsilon), (0. - epsilon, 1. + epsilon))
PI_perslay_parameters["variance_init"]       = rui(3.0, 3.0) 
PI_perslay_parameters["variance_const"]      = False
PI_perslay_parameters["cv_layers"]           = []
PI_perslay_parameters["persistence_weight"]  = "linear"
PI_perslay_parameters["coeff_init"]          = rui(1.0, 1.0)
PI_perslay_parameters["coeff_const"]         = False
PI_perslay_parameters["perm_op"]             = "sum"

Persistence landscape layer.

In [ ]:
LS_perslay_parameters                        = {}
LS_perslay_parameters["layer"]               = "ls"
LS_perslay_parameters["num_samples"]         = 100
LS_perslay_parameters["sample_init"]         = rui(0.0, 1.0) 
LS_perslay_parameters["sample_const"]        = False
LS_perslay_parameters["fc_layers"]           = []
LS_perslay_parameters["persistence_weight"]  = None
LS_perslay_parameters["perm_op"]             = "topk"
LS_perslay_parameters["keep"]                = 1

DeepSet layer.

In [ ]:
PM_perslay_parameters                           = {}
PM_perslay_parameters["layer"]                  = "pm"
PM_perslay_parameters["peq"]                    = [(100, "max")]
PM_perslay_parameters["weight_init"]            = rui(0.0, 1.0)
PM_perslay_parameters["weight_const"]           = False
PM_perslay_parameters["bias_init"]              = rui(0.0, 1.0) 
PM_perslay_parameters["bias_const"]             = False
PM_perslay_parameters["fc_layers"]              = []
PM_perslay_parameters["persistence_weight"]     = "grid"
PM_perslay_parameters["grid_size"]              = [20,20]
epsilon = .001
PM_perslay_parameters["grid_bnds"]              = ((0. - epsilon, 1. + epsilon), (0. - epsilon, 1. + epsilon))
PM_perslay_parameters["grid_init"]              = rui(0.0, .01)
PM_perslay_parameters["grid_const"]             = False
PM_perslay_parameters["perm_op"]                = "sum"

### Design the network

In the template below, we define a very simple `baseModel` that encodes a network architecture. In this model, we define a PersLay layer for each type of persistence diagrams (or filtration) used in input. This layer is a weighted average of several predefined PersLay layers if `combination` is set to `True`. Otherwise, there is one specific PersLay layer for each filtration. If you want all filtrations to share the same PersLay parameters, `perslay_parameters` must be a single dictionary containing the keys defined above. If you want filtration-specific parameters, `perslay_parameters` must be a list containing several such dictionaries. Eventual additional features are simply concatenated with the outputs of these PersLay channels, and a single fully-connected operation is then used to make the prediction.

In [ ]:
class baseModel:

    def __init__(self, filt_parameters, perslay_parameters, labels, combination=False): 
        self.filt_parameters = filt_parameters
        self.perslay_parameters = perslay_parameters
        self.num_labels = labels.shape[1]
        self.num_filts = len(self.filt_parameters["names"])
        self.combination = combination
        
    def get_parameters(self):
        return [self.filt_parameters, self.perslay_parameters, self.combination]

    def instance(self, indxs, feats, diags):
        
        if self.filt_parameters["learn"]:
            
            lpd = tf.load_op_library("persistence_diagram.so")
            hks = tf.load_op_library("hks.so")
            import _persistence_diagram_grad
            import _hks_grad
            
            H, T = np.array(self.filt_parameters["homology"]), np.array(self.filt_parameters["thresholds"])
            N, I = np.array([[self.num_filts]]), np.array(self.filt_parameters["init"], dtype=np.float32)
            cumsum = np.cumsum(np.array([0] + [thr for thr in T[:,0]]))
            times = tf.get_variable("times", initializer=I)
            conn  = hks.heat_kernel_signature(indxs, times)
            pdiag_array, _ = lpd.persistence_diagram(H, T, indxs, N, conn)
            pds = tf.reshape(pdiag_array, [-1, cumsum[-1], 3])
            pdiags  = [pds[:,cumsum[i]:cumsum[i+1],:] for i in range(self.num_filts)]
            
        else:
            pdiags = diags
            
        list_v = []
        
        if self.combination:
            
            n_pl = len(self.perslay_parameters)
            alpha = tf.get_variable("perslay_coeffs", initializer=np.array(np.ones(n_pl), dtype=np.float32))
                
            for i in range(self.num_filts):
            # A perslay channel must be defined for each type of persistence diagram. 
            # Here it is a linear combination of several pre-defined layers.
                
                list_dgm = []
                for prm in range(n_pl):
                    perslay_channel(output  =  list_dgm,              # list used to store all outputs
                                    name    =  "perslay-" + str(i),   # name of this layer
                                    diag    =  pdiags[i],             # i-th type of diagrams
                                    **self.perslay_parameters[prm])
            
                list_dgm = [tf.multiply(alpha[idx], tf.layers.batch_normalization(dgm)) 
                        for idx, dgm in enumerate(list_dgm)]
                list_v.append(tf.math.add_n(list_dgm))
        else:
            if type(self.perslay_parameters) is not list:
                for i in range(self.num_filts):
                # A perslay channel must be defined for each type of persistence diagram. 
                # Here they all have the same hyper-parameters.
                    perslay_channel(output  =  list_v,              # list used to store all outputs
                                    name    =  "perslay-" + str(i), # name of this layer
                                    diag    =  pdiags[i],           # i-th type of diagrams
                                    **self.perslay_parameters)
            else:
                for i in range(self.num_filts):
                # A perslay channel must be defined for each type of persistence diagram. 
                # Here they all have the same hyper-parameters.
                    perslay_channel(output  =  list_v,              # list used to store all outputs
                                    name    =  "perslay-" + str(i), # name of this layer
                                    diag    =  pdiags[i],           # i-th type of diagrams
                                    **self.perslay_parameters[i])

        # Concatenate all channels and add other features
        with tf.variable_scope("perslay"):
            representations = tf.concat(list_v, 1)
            representations = tf.layers.batch_normalization(representations)
        with tf.variable_scope("norm_feat"):
            feat = tf.layers.batch_normalization(feats)

        final_representations = tf.concat([representations, feat], 1)

        #  Final layer to make predictions
        with tf.variable_scope("final-dense"):
            logits = tf.layers.dense(final_representations, self.num_labels)

        return representations, logits

In [ ]:
model = baseModel(filt_parameters, perslay_parameters, labels, combination=False)

## Train the network

In each type of experiment for training the network (with either `single_run` or `perform_expe` function, see the sections below), you can either use precomputed diagrams with a predefined architecture, in which case the code will load (and print) the network parameters as described in [1] (choice of $\phi$, $w$...), optimizer (number of epochs, learning rate...), etc. and then use the persistence diagrams (and eventual features) that have been generated when calling `generate(dataset)` to feed the network.

### Single run

Using the `single_run` function means training the network once and observing the performance (classification accuracy) on the test set.
- For orbit datasets, we suggest to use a 70-30 train-test split, i.e. `test_size = 0.3`.
- For graph datasets, we suggest to use a 90-10 train-test split, i.e. `test_size = 0.1`.

In [ ]:
test_size = 0.1

#### Predefined persistence diagrams

Train and test accuracies are printed every 10 epochs during training. Note that (especially on small datasets like `"MUTAG", "COX2"` etc.), there can be an important variability in the accuracy reported on different calls of `single_run`.

In [ ]:
weight, times, vecs = single_run(test_size=test_size, path_dataset="", dataset="MUTAG", 
                                 visualize_weights_times=True,
                                 xmin=0., xmax=1., xstep=.001, ymin=0., ymax=1., ystep=.001)

#### Custom persistence diagrams

If you use your own diagrams and architecture, you have to specify optimization parameters. As for any neural network framework, PersLay benefits from the use of GPU(s). If a GPU is available (and `tensorflow-gpu` is installed), you can use it by setting `tower_type` to `"gpu"` in the dictionary below. If you even have more than 1 GPU available, you can specify this number in `num_tower`.

In [ ]:
optim_parameters = {"decay": 0., "learning_rate": 0.05, "num_epochs": 50, "tower_size": 128,
                    "optimizer": "adam", "epsilon": 1e-4, "num_tower": 1, "tower_type": "cpu"}

In case you just want to learn the best vectorization with PersLay, and then apply a standard classifier, run the following cell to define the standard classifiers and their parameters you want to cross-validate on.

In [ ]:
standard_parameters = [{"Estimator":         [RandomForestClassifier()]},
                       {"Estimator":         [SVC()],
                        "Estimator__kernel": ["linear", "rbf"], 
                        "Estimator__C":      [0.1, 1, 10]},
                       {"Estimator":         [AdaBoostClassifier()]}]

Perform single run. If `model` is a list containing several instances of `baseModel`, the best model among these is selected with cross validation. In that case, you can use `perslay_cv` to specify the number of folds you want to use. Note that if you want to cross validate on the standard classifiers defined above, you also have to specify `standard_cv` (the two cross validations---PersLay and standard classifiers---are independent) and set `standard_model` to `True`.

In [ ]:
weight, times = single_run(test_size=test_size, path_dataset=None, dataset="PROTEINS",
                           model=model, diags=diags, feats=feats, labels=labels, 
                           optim_parameters=optim_parameters, perslay_cv=10,
                           standard_model=True, standard_parameters=standard_parameters, standard_cv=10,
                           visualize_weights_times=True,
                           gmin=.8, gmax=1.2, xmin=0., xmax=1., xstep=.001, ymin=0., ymax=1., ystep=.001
                           )

### Full experiment with several runs

Using the `perform_expe` function means training the network with a $K$-fold validation scheme for several runs.

#### Predefined persistence diagrams.

In [ ]:
perform_expe(num_runs=1, path_dataset="", dataset="MUTAG")

#### Custom persistence diagrams.

In [ ]:
optim_parameters = {"decay": 0., "learning_rate": 0.01, "num_epochs": 50, "tower_size": 128, 
                    "optimizer": "adam", "epsilon": 1e-4, "num_tower": 1, "tower_type": "cpu",
                    "mode": "KF", "folds": 10}

In [ ]:
standard_parameters = [{"Estimator":         [RandomForestClassifier()]},
                       {"Estimator":         [SVC()],
                        "Estimator__kernel": ["linear", "rbf"], 
                        "Estimator__C":      [0.1, 1, 10]},
                       {"Estimator":         [AdaBoostClassifier()]}]

In [ ]:
perform_expe(num_runs=1, path_dataset=None, dataset="PROTEINS",
             model=model, diags=diags, feats=feats, labels=labels, 
             optim_parameters=optim_parameters, perslay_cv=10, 
             standard_model=True, standard_parameters=standard_parameters, standard_cv=10,
             verbose=False)

# Bibliography

[1] _PersLay: A Simple and Versatile Neural Network Layer for Persistence Diagrams._
Mathieu Carrière, Frederic Chazal, Yuichi Ike, Théo Lacombe, Martin Royer, Yuhei Umeda.

[2] _Deep Sets._
Manzil Zaheer, Satwik Kottur, Siamak Ravanbakhsh, Barnabas Poczos, Ruslan Salakhutdinov, Alexander Smola.
_Advances in Neural Information Processing Systems 30 (NIPS 2017)_

[3] _Learning Representations of Persistence Barcodes._
Christoph Hofer, Roland Kwitt, Marc Niethammer.
_JMLR (2019)_